# Risk-Aware Decision System with Abstention

This notebook demonstrates an end-to-end workflow for building a high-reliability ML system using the UCI Adult dataset.

### Key Features:
- **Probabilistic Modeling**: LightGBM with calibrated outputs.
- **Uncertainty Estimation**: Quantifying prediction confidence.
- **Decision Policy**: optimizing thresholds `tau_low`, `tau_high` to minimize asymmetric costs.
- **Evaluation**: Risk-Coverage analysis and Cost Reduction metrics.

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')

import matplotlib.pyplot as plt

from src.data_loader import load_and_preprocess
from src.model import train_probabilistic_model
from src.calibration import calibrate_model, apply_calibration, plot_reliability_diagram
from src.policy import optimize_thresholds, compute_cost
from src.evaluation import evaluate_policy, plot_risk_coverage_curve, plot_cost_reduction

## 1. Data Loading & Preprocessing
We load the Adult dataset, handle missing values, and perform one-hot encoding.

In [ ]:
data = load_and_preprocess(data_dir="../data")
X_train, y_train = data['X_train'], data['y_train']
X_val, y_val = data['X_val'], data['y_val']
X_test, y_test = data['X_test'], data['y_test']

print(f"Train size: {X_train.shape}")
print(f"Test size: {X_test.shape}")

## 2. Model Training
Train a LightGBM classifier to output raw probabilities.

In [ ]:
model = train_probabilistic_model(X_train, y_train, X_val, y_val)

## 3. Calibration
Models like Boosted Trees can be overconfident. We use Isotonic Regression on the Validation set to calibrate probabilities.

In [ ]:
val_probs_raw = model.predict(X_val)

fig, ax = plt.subplots(1, 2, figsize=(14, 5))
plot_reliability_diagram(y_val, val_probs_raw, title="Uncalibrated", ax=ax[0])

calibrator = calibrate_model(model, X_val, y_val)
val_probs_cal = apply_calibration(model, calibrator, X_val)

plot_reliability_diagram(y_val, val_probs_cal, title="Calibrated", ax=ax[1])
plt.show()

## 4. Policy Optimization
We minimize the expected cost:
- False Positive: 10
- False Negative: 3
- Abstain: 1

We search for `tau_low` and `tau_high`.

In [ ]:
tau_low, tau_high = optimize_thresholds(y_val, val_probs_cal)

## 5. Final Evaluation
Comparing our Risk-Aware System against a standard Baseline (threshold 0.5, no abstention).

In [ ]:
test_probs_cal = apply_calibration(model, calibrator, X_test)
metrics = evaluate_policy(y_test, test_probs_cal, tau_low, tau_high)

# Baseline cost
baseline_preds = (test_probs_cal >= 0.5).astype(int)
_, baseline_cost = compute_cost(y_test, baseline_preds, test_probs_cal)

print(f"Baseline Average Cost: {baseline_cost:.4f}")
print(f"Risk-Aware Average Cost: {metrics['Avg Cost']:.4f}")
print(f"Results: {metrics}")

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 6))
plot_risk_coverage_curve(y_test, test_probs_cal, ax=ax[0])
plot_cost_reduction(baseline_cost, metrics['Avg Cost'], ax=ax[1])
plt.show()